In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import sys
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler

from glob import glob
from skimage import io

np.random.seed(5)

In [ ]:
#collecting image path
#/content/drive/MyDrive/yolov5/yolov5/inference
output_text_path = '/content/drive/MyDrive/yolov5/runs/detect/exp/labels/*.txt'   
output_file = []
output_file.extend(sorted(glob(output_text_path)))

In [ ]:
print(f' output file : {len(output_file)} files')
# for file in output_file:
#   print(file)

 output file : 22 files


In [ ]:
#collecting image id

def get_ids(path_list):
  id_list = []
  for list in path_list:
    split_list = list.split('/')[-1].split('.')[0] + '.jpg'
    id_list.append(split_list)
  return id_list

output_id = get_ids(output_file)
print(output_id)

['12_C2C12.jpg', '13_Caco-2.jpg', '16_Caki.jpg', '17_Caki.jpg', '24_DLD.jpg', '2_769p.jpg', '37_HEL299.jpg', '39_HELA.jpg', '40_HELA.jpg', '43_hMSC.jpg', '44_HT1080.jpg', '45_HT1080.jpg', '52_MCC.jpg', '72_RAW.jpg', '75_RKO.jpg', '77_SH-SY5Y.jpg', '78_SH-SY5Y.jpg', '83_SW480.jpg', '91_Caki.jpg', '95_HCT116.jpg', '96_HELA.jpg', '98_HT1080.jpg']


In [ ]:
#read file
temp_FileContent = []
for file in output_file:
  file_name = file
  with open(file_name, "r") as ofile:
    temp_FileContent.append(ofile.read())
FileContent = []
for filecontent in temp_FileContent:
  filecontent = filecontent.rstrip(' \n')
  FileContent.append(filecontent)

In [ ]:
#collecting file content 
content_list = []
per_image_line = []
for content in FileContent:
  split_list = content.split('\n')
  per_image_line.append(len(split_list))
  for splitList in split_list:
    content_list.append(splitList)
# print(content_list)
print(len(content_list))
print(per_image_line)
print(len(per_image_line))

210
[11, 18, 13, 7, 40, 5, 6, 9, 6, 1, 1, 9, 15, 7, 5, 15, 6, 1, 13, 4, 4, 14]
22


In [ ]:
#making image id list
count = 0
image_id = []
for i in range(len(per_image_line)):
  for j in range(per_image_line[i]):
    image_id.append(output_id[i])

print(image_id)

['12_C2C12.jpg', '12_C2C12.jpg', '12_C2C12.jpg', '12_C2C12.jpg', '12_C2C12.jpg', '12_C2C12.jpg', '12_C2C12.jpg', '12_C2C12.jpg', '12_C2C12.jpg', '12_C2C12.jpg', '12_C2C12.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '13_Caco-2.jpg', '16_Caki.jpg', '16_Caki.jpg', '16_Caki.jpg', '16_Caki.jpg', '16_Caki.jpg', '16_Caki.jpg', '16_Caki.jpg', '16_Caki.jpg', '16_Caki.jpg', '16_Caki.jpg', '16_Caki.jpg', '16_Caki.jpg', '16_Caki.jpg', '17_Caki.jpg', '17_Caki.jpg', '17_Caki.jpg', '17_Caki.jpg', '17_Caki.jpg', '17_Caki.jpg', '17_Caki.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD.jpg', '24_DLD

In [ ]:
#making list of columns
category_id = []
confidence = []
x_center_norm = []
y_center_norm = []
width_norm = []
height_norm = []
for content in content_list:
  split_content = content.split(' ')
  # print(split_content)
  category_id.append(split_content[0])
  confidence.append(float(split_content[1]))
  x_center_norm.append(float(split_content[2]))
  y_center_norm.append(float(split_content[3]))
  width_norm.append(float(split_content[4]))
  height_norm.append(float(split_content[5]))
# print(len(image_id))
# print(len(class_id))
# print(len(image_id))
# print(len(image_id))
# print(len(image_id))

In [ ]:
#making dataframe
df = {'filename': image_id,
      'category_id': category_id,
      'score': confidence,
      'x_center_norm': x_center_norm,
      'y_center_norm': y_center_norm,
      'width_norm': width_norm,
      'height_norm': height_norm}
df = pd.DataFrame(df)
df

,filename,category_id,score,x_center_norm,y_center_norm,width_norm,height_norm
0,12_C2C12.jpg,21,0.035235,0.055272,0.070470,0.066326,0.102356
1,12_C2C12.jpg,12,0.072987,0.385204,0.095638,0.117347,0.117432
2,12_C2C12.jpg,35,0.507550,0.710034,0.716443,0.579932,0.138550
3,12_C2C12.jpg,1,0.072987,0.385204,0.095638,0.117347,0.149048
4,12_C2C12.jpg,35,0.537752,0.641156,0.300336,0.275510,0.151001
...,...,...,...,...,...,...,...
205,98_HT1080.jpg,5,0.321309,0.795337,0.132550,0.094991,0.217651
206,98_HT1080.jpg,49,0.692114,0.678756,0.615772,0.594128,0.224976
207,98_HT1080.jpg,1,0.375000,0.207254,0.058725,0.041451,0.233154
208,98_HT1080.jpg,1,0.071309,0.834197,0.065436,0.065630,0.479492


In [ ]:
category_csv = '/content/drive/MyDrive/category.csv'
df_cat = pd.read_csv(category_csv)
# df_cat

In [ ]:
#class id to class name
df['category_id'] = df['category_id'].astype(int) + 1
df['category_name'] = ''
for i in range(df['category_id'].size):
  df['category_name'].iloc[i] = df_cat['name'].iloc[df['category_id'].iloc[i]-1]
df

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,filename,category_id,score,x_center_norm,y_center_norm,width_norm,height_norm,category_name
0,12_C2C12.jpg,22,0.035235,0.055272,0.070470,0.066326,0.102356,hMSC_Nucleus
1,12_C2C12.jpg,13,0.072987,0.385204,0.095638,0.117347,0.117432,Hel299_Nucleus
2,12_C2C12.jpg,36,0.507550,0.710034,0.716443,0.579932,0.138550,ACHN
3,12_C2C12.jpg,2,0.072987,0.385204,0.095638,0.117347,0.149048,CHO_Nucleus
4,12_C2C12.jpg,36,0.537752,0.641156,0.300336,0.275510,0.151001,ACHN
...,...,...,...,...,...,...,...,...
205,98_HT1080.jpg,6,0.321309,0.795337,0.132550,0.094991,0.217651,ACHN_Nucleus
206,98_HT1080.jpg,50,0.692114,0.678756,0.615772,0.594128,0.224976,MG-63
207,98_HT1080.jpg,2,0.375000,0.207254,0.058725,0.041451,0.233154,CHO_Nucleus
208,98_HT1080.jpg,2,0.071309,0.834197,0.065436,0.065630,0.479492,CHO_Nucleus


In [ ]:
image_df = pd.read_csv('/content/drive/MyDrive/eval_difficult_images.csv')
# image_df

In [ ]:
df = pd.merge(df,image_df,on = 'filename', how = 'left')
df

In [ ]:
df['xmin'] = df['x_center_norm'] * df['width'] - df['width_norm'] * df['width']/2
df['ymin'] = df['y_center_norm'] * df['height'] - df['height_norm'] * df['height']/2
df['xmax'] = df['x_center_norm'] * df['width'] + df['width_norm'] * df['width']/2
df['ymax'] = df['y_center_norm'] * df['height'] + df['height_norm'] * df['height']/2

df = df.rename({'width': 'image_width', 'height': 'image_height', 'id': 'image_id'},axis = 1)

for i in range(df['xmin'].size):
  if df['xmin'].iloc[i] < 0:
    df['xmin'].iloc[i] = 0
  if df['ymin'].iloc[i] < 0:
    df['ymin'].iloc[i] = 0
  if df['xmax'].iloc[i] > df['image_width'].iloc[i]:
    df['xmax'].iloc[i] = df['image_width'].iloc[i]
  if df['ymax'].iloc[i] > df['image_height'].iloc[i]:
    df['ymax'].iloc[i] = df['image_height'].iloc[i]
  
df.drop(['x_center_norm','y_center_norm','width_norm','height_norm','license'],axis = 1,inplace = True)
df = df[['image_id','filename','category_id','category_name','score','xmin','ymin','xmax','ymax','image_height','image_width']]
df

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,image_id,filename,category_id,category_name,score,xmin,ymin,xmax,ymax,image_height,image_width
0,1,12_C2C12.jpg,22,hMSC_Nucleus,0.035235,13.176875,11.343578,52.707469,71.528906,588,596
1,1,12_C2C12.jpg,13,Hel299_Nucleus,0.072987,194.612178,21.709901,264.550990,90.759917,588,596
2,1,12_C2C12.jpg,36,ACHN,0.507550,250.360528,380.534784,596.000000,462.002184,588,596
3,1,12_C2C12.jpg,2,CHO_Nucleus,0.072987,194.612178,12.414797,264.550990,100.055021,588,596
4,1,12_C2C12.jpg,36,ACHN,0.537752,300.026996,132.203274,464.230956,220.991862,588,596
...,...,...,...,...,...,...,...,...,...,...,...
205,31,98_HT1080.jpg,6,ACHN_Nucleus,0.321309,445.713415,13.736485,502.328289,139.756415,579,596
206,31,98_HT1080.jpg,50,MG-63,0.692114,227.488432,291.401436,581.588720,421.662540,579,596
207,31,98_HT1080.jpg,2,CHO_Nucleus,0.375000,111.171046,0.000000,135.875722,101.499742,579,596
208,31,98_HT1080.jpg,2,CHO_Nucleus,0.071309,477.623553,0.000000,516.739271,176.700494,579,596


In [ ]:
df.to_csv('/content/drive/My Drive/predicted.csv',index = False)